In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
df = pd.read_csv('/content/mushrooms.csv')
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


We see that all the data is categorical data , let's transform the categorical data into numerical

In [9]:
from sklearn.preprocessing import LabelEncoder
new_df = df.apply(LabelEncoder().fit_transform)

In [10]:
new_df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,0,2,2,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,1,3,2,2,7,7,0,2,1,0,3,0,1


##Start of Neural network function

In [11]:
#Activation functions
def sigmoid(x):
  return 1/(1+np.exp(-x))

def sigmoid_derivative(x):
  return sigmoid(x)*(1-sigmoid(x))

def softmax(x):
    e_x = np.exp(x)
    softmax = e_x / e_x.sum(axis=1, keepdims=True)
    return softmax

def relu(x):
  return np.maximum(0,x)

def relu_derivative(x):
  x[x <=0] = 0
  x[x>0]=1
  return x

### Back Propogation

In [12]:
def back_prop(inputs, hw, ow, hb, ob, alpha, zh, ah, zo, diff):
    
    # output layer weights cost function
    dcost_dao = diff
    dao_dzo = sigmoid_derivative(zo) 
    dzo_dwo = ah
    dcost_wo = np.dot(dzo_dwo.T, dcost_dao * dao_dzo)
    
    # hidden layer weights cost function
    dcost_dzo = dcost_dao * dao_dzo
    dzo_dah = ow
    dcost_dah = np.dot(dcost_dzo , dzo_dah.T)
    dah_dzh = relu_derivative(zh) 
    dzh_dwh = inputs
    dcost_wh = np.dot(dzh_dwh.T, dah_dzh * dcost_dah)
    
    # update weights
    hw -= alpha * dcost_wh
    ow -= alpha * dcost_wo
    
    # update biases
    for d in dcost_wh:
        hb -= alpha * d
    
    for d in dcost_wo:
        ob -= alpha * d

### Feed Forward and Model Training




In [13]:
def mlp(inputs, outputs, hidden_layer_width, alpha, epochs):
    
    d = dict()
    np.random.seed(42)

    #hidden layers
    hw = np.random.rand(len(inputs[0]), hidden_layer_width)
    ow = np.random.rand(hidden_layer_width, 1)  

    # intialize biases for hidden and output layers to zeros
    hb = np.zeros((1, hidden_layer_width))
    ob = np.zeros((1, 1))

    # learning rate
    learning_rate = alpha


    for epoch in range(1, epochs + 1):
        # hidden layer
        zh = np.dot(inputs, hw) + hb
        ah = relu(zh)
    
        #feed forward using activation function
        zo = np.dot(ah, ow) + ob
        ao = softmax(zo)
    
        # calculate mean squared error
        diff = ao - outputs
        mse = (np.square(diff)).mean(axis=0)
        
        # backpropagation
        back_prop(inputs, hw, ow, hb, ob, alpha, zh, ah, zo, diff)
    
        # save the final weights and biases of the model for testing
        if epoch == epochs:
            print("After %d epochs, mean squared error is: %.2f" % (epoch,mse[0]))
            d['hidden layer weights'] = hw
            d['output layer weights'] = ow
            d['hidden layer biases'] = hb
            d['output layer biases'] = ob
            
    return d

### Model Accuracy

In [19]:
def model_accuracy(test_inputs, test_outputs, hidden_weights, output_weights, hidden_biases, output_biases):
    # hidden layer
    zh = np.dot(test_inputs, hidden_weights) + hidden_biases
    ah = relu(zh)
    
    # output layer
    zo = np.dot(ah, output_weights) + output_biases
    ao = softmax(zo)

    # test model accuracy

    tp, tn, fp, fn, pp = 0, 0, 0, 0,0

   

    correct = 0
    for i in range(len(test_outputs)):
        prediction = int(round(ao[i][0]))
        if prediction == 1:
            pp += 1

        actual = test_outputs[i][0]
        if prediction == actual:
            if actual == 1:
                tp += 1
            else:
                tn += 1
        else:
            if actual == 1:
                fn += 1
            else:
                fp += 1
        
    total = len(test_outputs)
    accuracy = (tp + tn) / total
    precision = tp / pp
  

    print("Accuracy is: %f" % accuracy)
    print("Precision is: %f" % precision)
  

### Train and Test for inputs in neural netowrk

In [15]:
# inputs and outputs
inputs = new_df.drop(columns=['class'])
outputs = new_df['class'].values

# 75-25 split for training and testing sets
split = round(.75 * len(inputs))

X_train = np.array(inputs[:split])
X_test = np.array(inputs[split:])

y_train = np.array(outputs[:split])
y_test = np.array(outputs[split:])

y_train = y_train.reshape(len(y_train),1)
y_test = y_test.reshape(len(y_test),1)

### Run model


In [16]:
model = mlp(inputs=X_train, outputs=y_train, hidden_layer_width=40, alpha=0.05, epochs=10000)

After 10000 epochs, mean squared error is: 0.60


### Model accuracy

In [20]:
hw = model['hidden layer weights']
ow = model['output layer weights']
hb = model['hidden layer biases']
ob = model['output layer biases']

model_accuracy(test_inputs=X_test, test_outputs=y_test, hidden_weights=hw, output_weights=ow, 
               hidden_biases=hb, output_biases=ob)

Accuracy is: 0.741507
Precision is: 0.741507
